In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from polarDensity_helper import Coord_Get, get_header_info
from site_distributions import make_simple_site, combine_sites, plot_density, make_symmetric_sites
from string import ascii_uppercase

# Step 0: define a site

This is done with the plotting_show_binID notebook.
Some examples from ELIC:

In [ ]:
lipid = "DPPC"
leaf = "upp"
data_root = Path("/home/ems363/Projects/ELIC_DPPC/elicPE17_DPPC/")
fpath = data_root.joinpath(f"{lipid}.dat.{leaf}.dat")

In [ ]:
num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
the_data = np.loadtxt(fpath, skiprows=1)
rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

In [ ]:
cheng_sites = make_symmetric_sites(the_data, 4, 3, 24, 30, Ntheta, dr, dth, exrho, frames, sitename="Cheng Site ")
cheng_site_symmetric = combine_sites(cheng_sites, exrho, "Cheng Sites, Symmetric", symmetric=True)
ulens_sites = make_symmetric_sites(the_data, 9, 4, 22, 32, Ntheta, dr, dth, exrho, frames, sitename="Ulens Site ")
#ulens_site = make_simple_site(the_data, 22, 32, 4, Ntheta, dr, dth, exrho, frames, np.arange(28,30,1), "Ulens Site")
ulens_site_symmetric = combine_sites(ulens_sites, exrho, "Ulens Sites, Symmetric", symmetric=True)

sites = {'cheng_site':cheng_sites[0], 
         'cheng_site_symmetric':cheng_site_symmetric,
         'ulens_site':ulens_sites[0],
         'ulens_site_symmetric':ulens_site_symmetric
         }

# Step 1 get the total area of the site in question

In [ ]:
for name, site in sites.items():
    print(f"{name:<20}:{np.round(site.area,1):>5}")

# Step 2 get the density 
In VMD:
1. load the trajectory of the empty membrane
2. modify do_get_counts.tcl to have the total area of the site
3. source do_get_counts.tcl
4. It will save to a file called counts_[area].out
5. Provide the path here:

In [ ]:
area = 125
bulk_counts_path = Path(f"/home/ems363/Projects/Polar_Binning_DeltaG/accessible_area/counts_{area}.out")
bulk_counts = np.loadtxt(bulk_counts_path)

frequencies = np.bincount(bulk_counts.astype(int).flatten())
probabilities = frequencies/np.sum(frequencies)
fig, ax = plt.subplots()
ax.plot(range(len(probabilities)), probabilities)
ax.set_ylabel("Probability")
ax.set_xlabel(f"Number of beads in an area about {area} "+r"$\AA^2$")
bulk_mode = np.argmax(probabilities)
ax.vlines([bulk_mode], 0, np.max(probabilities), color = 'black', linestyles='dashed', label = f"mode={bulk_mode}")
ax.legend()

# Compare the bulk mode with the site mode

In [ ]:
rows = 2
cols = 2
fig, axes = plt.subplots(rows,cols, figsize=(15,15))

for site, ax in zip(sites.values(), axes.flatten()):
    ax = plot_density(site, ax)

plt.savefig(data_root.joinpath("Raw_distributions.pdf"))

In [ ]:
print(f"{'Site Name':<20}:{'Experimental Mode':>5}")
expt_modes = {}
for key, site in sites.items():
    counts = site.counts.astype(int)
    bincounts = np.bincount(counts.flatten())[1:]
    the_mode = np.argmax(bincounts)
    expt_modes[key] = the_mode
    print(f"{key:<20}:{np.round(the_mode,1):>5}")

# The accessible area is...
ASSUMING $\left(A_\mathrm{site} \approx A_\mathrm{bulk}\right) \forall A_\mathrm{site}$

$A_\mathrm{acc} = A_\mathrm{total} \frac{M_\mathrm{site}}{M_\mathrm{bulk}}$

In [ ]:
print(f"{'Site Name':<20}:{'Accessible Area':>5}")
areas = {}
for name, site in sites.items():
    A_acc = site.area * expt_modes[name] / bulk_mode
    site.area = A_acc
    print(f"{name:<20}:{np.round(A_acc,1):>5}")

# Replot with new areas:

In [ ]:
for name, site in sites.items():
    site.Npeak = exrho* site.area

In [ ]:
rows = 2
cols = 2
fig, axes = plt.subplots(rows,cols, figsize=(15,15))

for site, ax in zip(sites.values(), axes.flatten()):
    ax = plot_density(site, ax)
    ax.text(.01, .99, f"Accessible area:"+str(np.round(site.area,1))+r"$\AA^2$", ha='left', va='top', transform=ax.transAxes)

plt.savefig(data_root.joinpath(f"replotted_with_corrected_expectation_area{area}_{leaf}.pdf"))